In [ ]:
import os                       # for working with files
import numpy as np              # for numerical computationss
import pandas as pd             # for working with dataframes
import torch                    # Pytorch module 
import matplotlib.pyplot as plt # for plotting informations on graph and images using tensors
import torch.nn as nn           # for creating  neural networks
from torch.utils.data import DataLoader # for dataloaders 
from PIL import Image           # for checking images
import torch.nn.functional as F # for functions for calculating loss
import torchvision.transforms as transforms   # for transforming images into tensors 
from torchvision.utils import make_grid       # for data checking
from torchvision.datasets import ImageFolder  # for working with classes and images
from torchsummary import summary              # for getting the summary of our model

%matplotlib inline

In [ ]:
data_dir = "data"

In [ ]:
train_dir = "train/"
valid_dir = "valid/"
diseases = os.listdir(train_dir)

In [ ]:
print("Total disease classes are: {}".format(len(diseases)))

In [ ]:
plants = []
NumberOfDiseases = 0
for plant in diseases:
    if plant.split('___')[0] not in plants:
        plants.append(plant.split('___')[0])
    if plant.split('___')[1] != 'healthy':
        NumberOfDiseases += 1

In [ ]:
n_train = 0
for value in nums.values():
    n_train += value
print(f"There are {n_train} images for training")

In [ ]:
# datasets for training
train = ImageFolder(train_dir, transform=transforms.ToTensor())
valid = ImageFolder(valid_dir, transform=transforms.ToTensor()) 

In [ ]:
img, label = train[0]
print(img.shape, label)

In [ ]:
def show_image(image, label):
    print("Label :" + train.classes[label] + "(" + str(label) + ")")
    plt.imshow(image.permute(1, 2, 0))

In [ ]:
random_seed = 7
torch.manual_seed(random_seed)

In [ ]:
# setting the batch size
batch_size = 32

In [ ]:
# DataLoaders for training and validation
train_dl = DataLoader(train, batch_size, shuffle=True, num_workers=2, pin_memory=True)
valid_dl = DataLoader(valid, batch_size, num_workers=2, pin_memory=True)

In [ ]:
def show_batch(data):
    for images, labels in data:
        fig, ax = plt.subplots(figsize=(30, 30))
        ax.set_xticks([]); ax.set_yticks([])
        ax.imshow(make_grid(images, nrow=8).permute(1, 2, 0))
        break

In [ ]:
show_batch(train_dl) 

In [ ]:
# for moving data to device (CPU or GPU)
def to_device(data, device):
    """Move tensor(s) to chosen device"""
    if isinstance(data, (list,tuple)):
        return [to_device(x, device) for x in data]
    return data.to(device, non_blocking=True)

# for loading in the device (GPU if available else CPU)
class DeviceDataLoader():
    """Wrap a dataloader to move data to a device"""
    def __init__(self, dl, device):
        self.dl = dl
        self.device = device
        
    def __iter__(self):
        """Yield a batch of data after moving it to device"""
        for b in self.dl:
            yield to_device(b, self.device)
        
    def __len__(self):
        """Number of batches"""
        return len(self.dl)

In [ ]:
device = torch.device("cpu")
device

In [ ]:
from pylab import *
import torch.optim as optim
from torch.optim import lr_scheduler
from torchvision import datasets, models, transforms
import copy
from copy import deepcopy
import torchvision.transforms

In [ ]:
transforms_ = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(256),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'valid': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])
}

# Import the dataset
data_dir = "data"
image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x), transform=transforms_[x])
                  for x in ['train', 'valid']}

# Shuffle the dataset and create batches from the dataset
dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], shuffle=True, num_workers=4)
               for x in ['train', 'valid']}

# Get the number of images in each folder
data_size = {x: len(image_datasets[x]) for x in ['train', 'valid']}

# Get the class name
class_names = image_datasets['train'].classes

# Enable the GPU if it exists
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [ ]:
model_vgg16 = models.vgg16(pretrained=True)
model_alexnet = models.alexnet(pretrained=True)

# Define the function to train the model
def train_model(model, criterion, optimizer, scheduler, num_epochs=25):
    since = time.time()
    
    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0
    
    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)
        
        for phase in ['train', 'valid']:
            if phase == 'train':
                model.train()
            else:
                model.eval()
                
            running_loss = 0.0
            running_corrects = 0
            
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)
                
                optimizer.zero_grad()
                
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)
                    
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()
                        
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
                
            if phase == 'train':
                scheduler.step()
                
            epoch_loss = running_loss / data_size[phase]
            epoch_acc = running_corrects.double() / data_size[phase]
            
            print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                phase, epoch_loss, epoch_acc))
            
            if phase == 'valid' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())
        print()
    
    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))
    
    model.load_state_dict(best_model_wts)
    return model

# Set the requires_grad on each parameter to false, 
# so it will not calculate the gradients
for param in model_vgg16.parameters():
    param.requires_grad = False

for param in model_alexnet.parameters():
    param.requires_grad = False

# Set the new final layer for our new dataset
num_ftrs = model_vgg16.classifier[6].in_features
model_vgg16.classifier[6] = nn.Linear(num_ftrs, len(class_names))

num_ftrs = model_alexnet.classifier[6].in_features
model_alexnet.classifier[6] = nn.Linear(num_ftrs, len(class_names))

# Enable GPU for the model
model_vgg16 = model_vgg16.to(device)
model_alexnet = model_alexnet.to(device)

# Set the loss function
criterion = nn.CrossEntropyLoss()

In [ ]:
def call_Alex(opt_func,model,model_name):
    optimizer_conv = opt_func(model, lr=0.01)
    exp_lr_scheduler = lr_scheduler.StepLR(optimizer_conv, step_size=7)
    model_alexnet = train_model(model_name, criterion, optimizer_conv, exp_lr_scheduler, num_epochs=15)
    
    
def call_VGG(opt_func,model,model_name):
    optimizer_conv = opt_func(model, lr=0.01)
    exp_lr_scheduler = lr_scheduler.StepLR(optimizer_conv, step_size=7)
    model_alexnet = train_model(model_name, criterion, optimizer_conv, exp_lr_scheduler, num_epochs=15)

In [ ]:
list2=[torch.optim.ASGD]
print("---------Vgg16-----------")
for i in list2:
    print(i)
    PATH = "./"+str(i)+"_VGG_final_state.pth"  
    torch.save(model_vgg16.state_dict(), PATH)
    PATH = "./"+str(i)+"_VGG_final.pth"
    torch.save(model_vgg16, PATH)
    call_VGG(i,model_vgg16.classifier[6].parameters(),model_vgg16.)